In [137]:
import pandas as pd
import pickle
from scipy.stats import poisson
import os
from string import ascii_uppercase as alphabet

In [138]:
os.chdir(r'C:\Users\Ananya\PycharmProjects\Euros Prediction')

In [139]:
dict_table = pickle.load(open('dict_table', 'rb'))


In [140]:
import os
os.getcwd()
os.chdir(r'C:\Users\Ananya\AppData\Roaming\JetBrains\PyCharm2024.1\scratches')

In [141]:
df_UEFA = pd.read_csv('clean_UEFA_last_10_year_matches.csv')
df_fixture = pd.read_csv('clean_UEFA_Euro_2024_fixture.csv')

In [142]:
df_home = df_UEFA[['home', 'HomeGoals', 'AwayGoals']]
df_away = df_UEFA[['away', 'HomeGoals', 'AwayGoals']]

In [143]:
df_home = df_home.rename(columns={'home': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns = {'away': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

In [144]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x==y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        points_home = 3 * (prob_home + prob_draw)
        points_away = 3 * (prob_away + prob_draw)
        return points_home, points_away
    else:
        return 0, 0

In [145]:
df_fixture_group_36 = df_fixture[:36].copy()
df_fixture_group_ro16 = df_fixture[36:44].copy()
df_fixture_group_quarters = df_fixture[44:48].copy()
df_fixture_group_semis = df_fixture[48:50].copy()
df_fixture_group_final = df_fixture[50:].copy()


In [146]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_36[df_fixture_group_36['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

C:\Users\Ananya\AppData\Local\Temp\ipykernel_25992\62853046.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[5.22414904]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Ananya\AppData\Local\Temp\ipykernel_25992\62853046.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[5.57755623]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Ananya\AppData\Local\Temp\ipykernel_25992\62853046.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.40198341]' has dtype incompatible with int64, pl

In [147]:
dict_table['Group F'] 



,Team,Pts
0,Turkey,9.0
1,Portugal,8.0
2,Czech Republic,5.0
3,Georgia,3.0


In [148]:
df_fixture_group_ro16

,home,score,away,Tournament
36,Runner-up Group A,Match 38,Runner-up Group B,UEFA_Euro_2024
37,Winner Group A,Match 37,Runner-up Group C,UEFA_Euro_2024
38,Winner Group C,Match 40,3rd Group D/E/F,UEFA_Euro_2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,UEFA_Euro_2024
40,Runner-up Group D,Match 42,Runner-up Group E,UEFA_Euro_2024
41,Winner Group F,Match 41,3rd Group A/B/C,UEFA_Euro_2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,UEFA_Euro_2024
43,Winner Group D,Match 44,Runner-up Group F,UEFA_Euro_2024


In [149]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_group_ro16.replace({f'Winner {group}':group_winner,
                                 f'Runner-up {group}':runners_up}, inplace=True)

df_fixture_group_ro16

,home,score,away,Tournament
36,Switzerland,Match 38,Italy,UEFA_Euro_2024
37,Germany,Match 37,Denmark,UEFA_Euro_2024
38,England,Match 40,3rd Group D/E/F,UEFA_Euro_2024
39,Spain,Match 39,3rd Group A/D/E/F,UEFA_Euro_2024
40,Netherlands,Match 42,Romania,UEFA_Euro_2024
41,Turkey,Match 41,3rd Group A/B/C,UEFA_Euro_2024
42,Belgium,Match 43,3rd Group A/B/C/D,UEFA_Euro_2024
43,France,Match 44,Portugal,UEFA_Euro_2024


In [150]:
group_name = []
for letter, i in zip(alphabet, range(0,6,1)):
    group_name.append(
        [f'Group {letter}']
    )
group_names = pd.DataFrame(group_name)
third_team = []
for group in dict_table:
    third_team.append(
        dict_table[group].loc[2, 'Team']
    )
third_teams = pd.DataFrame(third_team)    
third_team_pts = []
for group in dict_table:
    third_team_pts.append(
        dict_table[group].loc[2, 'Pts']
    )
third_team_pt = pd.DataFrame(third_team_pts)   
third_team_tab = group_names.join(third_teams, how='left', lsuffix='_left', rsuffix='_right')
third_team_table = third_team_tab.join(third_team_pt, how='right', lsuffix='_left', rsuffix='_right')
third_team_table.columns = ['Groups', 'Teams', 'Pts']
third_team_table = third_team_table.sort_values(['Pts'], ascending=[False]).reset_index(drop=True)

third_team_table = third_team_table.drop(4)
third_team_table = third_team_table.drop(5)
third_team_table = third_team_table.sort_values(['Groups'], ascending=[True]).reset_index(drop=True)
  

In [151]:

third_team_table

,Groups,Teams,Pts
0,Group A,Scotland,5.0
1,Group C,Slovenia,5.0
2,Group E,Slovakia,7.0
3,Group F,Czech Republic,5.0


In [152]:
if third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group B' and third_team_table.loc[2, 'Groups'] == 'Group C' and third_team_table.loc[3, 'Groups'] == 'Group D':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[0, 'Teams']
    third_group_three = third_team_table.loc[2, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group B' and third_team_table.loc[2, 'Groups'] == 'Group C' and third_team_table.loc[3, 'Groups'] == 'Group E':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[0, 'Teams']
    third_group_three = third_team_table.loc[2, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group B' and third_team_table.loc[2, 'Groups'] == 'Group C' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[0, 'Teams']
    third_group_three = third_team_table.loc[2, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group B' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group E':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[2, 'Teams']
    third_group_three = third_team_table.loc[1, 'Teams']
    third_group_four = third_team_table.loc[0, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group B' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[2, 'Teams']
    third_group_three = third_team_table.loc[1, 'Teams']
    third_group_four = third_team_table.loc[0, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group E':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group E' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[2, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group A' and third_team_table.loc[1, 'Groups'] == 'Group D' and third_team_table.loc[2, 'Groups'] == 'Group E' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[3, 'Teams']
    third_group_two = third_team_table.loc[2, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group B' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group E':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[1, 'Teams']
    third_group_four = third_team_table.loc[0, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group B' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group D' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group B' and third_team_table.loc[1, 'Groups'] == 'Group C' and third_team_table.loc[2, 'Groups'] == 'Group E' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group B' and third_team_table.loc[1, 'Groups'] == 'Group D' and third_team_table.loc[2, 'Groups'] == 'Group E' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
elif third_team_table.loc[0, 'Groups'] == 'Group C' and third_team_table.loc[1, 'Groups'] == 'Group D' and third_team_table.loc[2, 'Groups'] == 'Group E' and third_team_table.loc[3, 'Groups'] == 'Group F':
    third_group_one = third_team_table.loc[2, 'Teams']
    third_group_two = third_team_table.loc[3, 'Teams']
    third_group_three = third_team_table.loc[0, 'Teams']
    third_group_four = third_team_table.loc[1, 'Teams']
df_fixture_group_ro16 = df_fixture_group_ro16.replace({f'3rd Group D/E/F':third_group_one, f'3rd Group A/D/E/F':third_group_two, f'3rd Group A/B/C':third_group_three, f'3rd Group A/B/C/D':third_group_four})

df_fixture_group_ro16

,home,score,away,Tournament
36,Switzerland,Match 38,Italy,UEFA_Euro_2024
37,Germany,Match 37,Denmark,UEFA_Euro_2024
38,England,Match 40,Czech Republic,UEFA_Euro_2024
39,Spain,Match 39,Slovakia,UEFA_Euro_2024
40,Netherlands,Match 42,Romania,UEFA_Euro_2024
41,Turkey,Match 41,Scotland,UEFA_Euro_2024
42,Belgium,Match 43,Slovenia,UEFA_Euro_2024
43,France,Match 44,Portugal,UEFA_Euro_2024


In [153]:
df_fixture_group_ro16 ['winner'] = '?'

In [154]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner 
    return df_fixture_updated

In [155]:
get_winner(df_fixture_group_ro16)

,home,score,away,Tournament,winner
36,Switzerland,Match 38,Italy,UEFA_Euro_2024,Italy
37,Germany,Match 37,Denmark,UEFA_Euro_2024,Denmark
38,England,Match 40,Czech Republic,UEFA_Euro_2024,England
39,Spain,Match 39,Slovakia,UEFA_Euro_2024,Spain
40,Netherlands,Match 42,Romania,UEFA_Euro_2024,Netherlands
41,Turkey,Match 41,Scotland,UEFA_Euro_2024,Turkey
42,Belgium,Match 43,Slovenia,UEFA_Euro_2024,Belgium
43,France,Match 44,Portugal,UEFA_Euro_2024,Portugal


In [156]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winner {match}': winner}, inplace=True)
    df_fixture_round_2 ['winner'] = '?'
    return df_fixture_round_2

In [157]:
update_table(df_fixture_group_ro16, df_fixture_group_quarters)

,home,score,away,Tournament,winner
44,Spain,Match 45,Denmark,UEFA_Euro_2024,?
45,Turkey,Match 46,Netherlands,UEFA_Euro_2024,?
46,England,Match 48,Italy,UEFA_Euro_2024,?
47,Belgium,Match 47,Portugal,UEFA_Euro_2024,?


In [158]:
get_winner(df_fixture_group_quarters)

,home,score,away,Tournament,winner
44,Spain,Match 45,Denmark,UEFA_Euro_2024,Spain
45,Turkey,Match 46,Netherlands,UEFA_Euro_2024,Netherlands
46,England,Match 48,Italy,UEFA_Euro_2024,England
47,Belgium,Match 47,Portugal,UEFA_Euro_2024,Portugal


In [159]:
update_table(df_fixture_group_quarters, df_fixture_group_semis)

,home,score,away,Tournament,winner
48,Spain,Match 49,Netherlands,UEFA_Euro_2024,?
49,Portugal,Match 50,England,UEFA_Euro_2024,?


In [160]:
get_winner(df_fixture_group_semis)

,home,score,away,Tournament,winner
48,Spain,Match 49,Netherlands,UEFA_Euro_2024,Spain
49,Portugal,Match 50,England,UEFA_Euro_2024,Portugal


In [161]:
update_table(df_fixture_group_semis, df_fixture_group_final)


,home,score,away,Tournament,winner
50,Spain,Match 51,Portugal,UEFA_Euro_2024,?


In [162]:
get_winner(df_fixture_group_final)

,home,score,away,Tournament,winner
50,Spain,Match 51,Portugal,UEFA_Euro_2024,Spain
